### Author: Kubam Ivo
### Date: 8/12/2020
### Purpose: Titanic Kaggle competition

### Logistic Regression

Using logistic regression as the baseline model, the solver parmater was specified as liblinear due to the size of the train dataset. The other parameters for this dataset were kept to their defaults. Due to the size of the train dataset, five fold cross validation was used to determine key metrics. This model gave a auc of 0.78, accuracy of 0.8 and weighted average f1-score of 0.8. The missing age values were imputed using the same regression technique used for the train dataset during preprocessing. The one Fare missing value was imputed using the mean value of fare of 32.2. The Kaggle value for the logistic regression was 0.76794

In [39]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


In [31]:
#Importing the clean dataset
train_data = pd.read_csv("train_clean1")
train_data.head()

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,1,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,0,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


In [32]:
# Importing the test data set
test_data = pd.read_csv("test.csv")

In [33]:
# Extracting the features
X = train_data.iloc[:,1:]
#Extracting the labels
y = train_data["Survived"]

In [34]:
#Initialising model class
logistic = LogisticRegression(solver='liblinear')

In [35]:
# 5 fold cross validation
y_pred = cross_val_predict(logistic, X, y, cv=5)

In [36]:
# Confusion matrix
confusion_matrix(y,y_pred, labels=[0, 1])

array([[476,  73],
       [103, 239]], dtype=int64)

In [37]:
#AUC
roc_auc_score(y,y_pred)

0.7829306873741732

In [40]:
#Accuracy
accuracy_score(y, y_pred)

0.8024691358024691

In [38]:
print(classification_report(y, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.82      0.87      0.84       549
           1       0.77      0.70      0.73       342

   micro avg       0.80      0.80      0.80       891
   macro avg       0.79      0.78      0.79       891
weighted avg       0.80      0.80      0.80       891



In [21]:
# Fitting the logistic regression model
logistic.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [22]:
test_data1 = pd.get_dummies(test_data,columns=["Pclass","Sex","Embarked"])


In [23]:
test_data1 = test_data1.drop(["PassengerId","Name", "Ticket", "Cabin"], axis=1)


In [24]:
test_data1.isnull().sum()

Age           86
SibSp          0
Parch          0
Fare           1
Pclass_1       0
Pclass_2       0
Pclass_3       0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

In [25]:
# Handling missing values
#test_data1["Age"] = test_data1["Age"].fillna(29.7) # imputing the mean value of 29.7 for all missing ages
test_data1["Fare"] = test_data1["Fare"].fillna(32.2)

# Handling missing values in age column usng linear regression
from sklearn.linear_model import LinearRegression

# Extracting the features
X = train_data.iloc[:,2:]
#Extracting the labels
y = train_data["Age"]

reg = LinearRegression().fit(X,y)
print(reg.score(X,y))


x1 = pd.isnull(test_data1["Age"]) # Extracting rows with null values for Age
x2 = test_data1[x1].index #Extracting index for all rows with null values for age
x_test = test_data1.iloc[x2,1:] # extracting test dataset where age is null
y_pred = reg.predict(x_test) #Predicting Age
test_data1.iloc[x2,0]=y_pred #replacing all null ages from  original dataset with predicted values
test_data1.isnull().sum()


0.32237586235082505


Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [26]:
test_data1.head(5)

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,34.5,0,0,7.8292,0,0,1,0,1,0,1,0
1,47.0,1,0,7.0000,0,0,1,1,0,0,0,1
2,62.0,0,0,9.6875,0,1,0,0,1,0,1,0
3,27.0,0,0,8.6625,0,0,1,0,1,0,0,1
4,22.0,1,1,12.2875,0,0,1,1,0,0,0,1


In [27]:
pred = logistic.predict(test_data1)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': pred})
output.to_csv('my_submissionLogistic.csv', index=False)